In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from keras import layers

In [28]:
#load Datasets
df = pd.read_csv("../datasets/telco.csv")

In [29]:
df.head()

,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,Stayed
1,1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,Stayed
2,2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Churned
3,3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,Stayed
4,4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Churned


In [30]:
# Preprocess
df.drop("customerID", axis=1, inplace=True)
df["Churn"] = df["Churn"].map({"Stayed": 0, "Churned": 1})
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)
print(df["Churn"])

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_7236\1387420261.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)


In [31]:
# One-hot encode categorical features
df = pd.get_dummies(df, drop_first=True)
print(df)

      Unnamed: 0  tenure  MonthlyCharges  TotalCharges  Churn  gender_Male  \
0              0       1           29.85         29.85      0        False   
1              1      34           56.95       1889.50      0         True   
2              2       2           53.85        108.15      1         True   
3              3      45           42.30       1840.75      0         True   
4              4       2           70.70        151.65      1        False   
...          ...     ...             ...           ...    ...          ...   
7038        7038      24           84.80       1990.50      0         True   
7039        7039      72          103.20       7362.90      0        False   
7040        7040      11           29.60        346.45      0        False   
7041        7041       4           74.40        306.60      1         True   
7042        7042      66          105.65       6844.50      0         True   

      SeniorCitizen_Yes  Partner_Yes  Dependents_Yes  PhoneServ

In [34]:
# Split
X = df.drop("Churn", axis=1)
y = df["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [35]:
# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
# Build model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# Train
history = model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=64, verbose=1)

Epoch 1/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.7291 - loss: 0.5410 - val_accuracy: 0.7817 - val_loss: 0.4593
Epoch 2/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7821 - loss: 0.4545 - val_accuracy: 0.7799 - val_loss: 0.4512
Epoch 3/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7929 - loss: 0.4499 - val_accuracy: 0.7808 - val_loss: 0.4461
Epoch 4/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7939 - loss: 0.4364 - val_accuracy: 0.7835 - val_loss: 0.4412
Epoch 5/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8001 - loss: 0.4249 - val_accuracy: 0.7879 - val_loss: 0.4407
Epoch 6/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7941 - loss: 0.4279 - val_accuracy: 0.7835 - val_loss: 0.4402
Epoch 7/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8018 - loss: 0.4283 - val_accuracy: 0.7897 - val_loss: 0.4388
Epoch 8/30
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8052 - loss: 0.4119 - val_accuracy: 0.7835 - val_loss

In [38]:
# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.4f}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7845 - loss: 0.4285 
✅ Test Accuracy: 0.7906


In [41]:
# Save model
model.save("../models/churn_model.h5")